In [1]:
#导语：我们可以把函数（当作object）赋给变量，如下把now赋给f，再调用f就是调用now
def now():
    print('2015-3-25')
f = now
f()

2015-3-25


In [11]:
#现在，假设我们要增强now()函数的功能，比如，在函数调用前后自动打印日志，但又不希望修改now()函数的定义，这种在代码运行期间动态增加功能的方式，称之为“装饰器”（Decorator）。
#简而言之，装饰器是把想要添加功能的函数nest进来
def log(func):#作为decorator，接受函数作为输入，再返回一个函数
    def wrapper(*args, **kw):#可接受任何参数输入
        print('call %s():' % func.__name__)#相当于matlab disp，func.__name__显示输入函数的name
        return func(*args, **kw)
    return wrapper#返回函数，闭包
@log#我们要借助Python的@语法，把decorator置于函数的定义处。 把@log放到now()函数的定义处，相当于执行了语句log(new)
def now():
    print('2015-3-25')
now()
now.__name__#值得注意，这里now的name改变了，因为我们返回的是warper
#由于log()是一个decorator，返回一个函数，所以，原来的now()函数仍然存在，只是现在同名的now变量指向了新的函数，
#于是调用now()将执行新函数，即在log()函数中返回的wrapper()函数。

call now():
2015-3-25


'wrapper'

In [12]:
#如果decorator本身需要传入参数，那就需要编写一个返回decorator的高阶函数，写出来会更复杂。比如，要自定义log的文本：
def log(text):
    def decorator(func):
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)#注意这三个return
        return wrapper
    return decorator
@log('execute')#当log需要输入时
def now():
    print('2015-3-25')
now()
now.__name__#值得注意，这里now的name改变了，因为我们返回了warper。
#这时，需要把原始函数的__name__等属性复制到wrapper()函数中，否则，有些依赖函数签名的代码执行就会出错
#不需要编写wrapper.__name__ = func.__name__这样的代码，使用Python内置的functools.wraps，见下面

execute now():
2015-3-25


'wrapper'

In [13]:
import functools

def log(text):
    def decorator(func):
        @functools.wraps(func)#保留原函数的签名,把func赋给wrap
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator
@log('execute')#当log需要输入时
def now():
    print('2015-3-25')
now()
now.__name__#这里是now了，需要依赖函数签名的情况下不改变

execute now():
2015-3-25


'now'

In [45]:
#test: 请设计一个decorator，它可作用于任何函数上，并打印该函数的执行时间：

import time, functools
def metric(fn):
    @functools.wraps(fn)#注意，一定要在warper（被赋予原函数签名的函数）前运行，否则会报错Invalid syntax。并且，这里要用wraps，不用函数名
    def wrapper(*args, **kw):
        a = time.time ()
        ccc = fn(*args, **kw)
        b = time.time ()
        print('%s executed in %s ms' % (fn.__name__, b-a))#print('%s executed in %.4f ms' % (fn.__name__, time.time() - t0))取四位
        return ccc
    return wrapper

"""
def metric(fn):
    @functools.wraps(fn)
    def wrapper(*args, **kw):
        t0 = time.time()
        result = fn(*args, **kw)
        print('%s executed in %.4f ms' % (fn.__name__, time.time() - t0))
        return result
    return wrapper
"""


              
# 测试
@metric
def fast(x, y):
    time.sleep(0.0012)
    return x + y;

@metric
def slow(x, y, z):
    time.sleep(0.1234)
    return x * y * z;

f = fast(11, 22)
s = slow(11, 22, 33)
if f != 33:
    print('测试失败!')
elif s != 7986:
    print('测试失败!')

fast executed in 0.003011465072631836 ms
slow executed in 0.12406492233276367 ms


In [46]:
#decorator，用函数作为输入。在使用时，在像使用的函数前调用。

fast executed in 0.0030 ms
slow executed in 0.1244 ms
